# Motivation and Overview of Data:

## Project Purpose:

There is alot of unknown flexability when it comes to recipes.  When you open up a recipe in a book, it is hard to know what leaneancy you have.

We want to compare recipies of different types to find trends in their nutritional value.  We hope that by creating a 

## Questions to Answer:


## Background knowledge/resources: 
Talk about what has been done, and what this will cover that is different.

this is a secret edit

## Data overview:
Talk about what the data looks like before we talk about how we got it (don't walk them through the cleaning process; walk them through what they will see, then talk about how we got it/cleaned it)

# Data Collection and Cleaning:

## Collection Procedure:
talk about how data is used at each step:

The recipe website divided its recipies into categories. Some examples of these are: Chili, Lamb, Potatoes, and even Christmas recipes!
The code to scrape this website divides into 4 functions.

- first one gets the links on the main site
- Then we separate them into categories
- We then collect the recipes
- And we parse through them

![title](example_recipe.png)

Now that we have the individual ingredients we can start collecting nutritional data

The scaper we build for this task searches each ingredient on nutritionvalue.org and takes the first available 3 links. 
Ex: Oats
![title](nutrition_value_query.png)

In [ ]:
## CODE: Scraper 1
def nutrition_website(ingredients):
    """Use Selenium to enter the given search query into the search bar of
    nutrion website and gets links to scrape data

    Returns:
        (dictonary): urls .
    """
    #initialize variables and chrome
    ingredients_dictionary = {}
    browser = webdriver.Chrome()
    browser.get("https://www.nutritionvalue.org/")
    num_links = 3
    try:
        for i in ingredients:
            try:
                #navigate
                search_bar = browser.find_element_by_name('food_query')
                search_bar.clear()
                search_bar.send_keys(Keys.CONTROL + "a")
                search_bar.send_keys(Keys.DELETE)
                search_bar.send_keys(i)

                search_bar.send_keys(Keys.RETURN)
                
                words = i.split()
                x = ""
                for n,w in enumerate(words):
                    if n == 0:
                        x += ".*(?<!food_query=)"+w+".*|"
                    else:
                        x += ".*(?<!\+)"+w+".*|"
                x = x[:-1]
                find = re.compile(x,re.IGNORECASE)
            
                
                # wait for page to load
                time.sleep(2)
                currentURL = browser.current_url
                if "food_query" in currentURL:
                    
                    links = browser.find_elements_by_tag_name('a')
                    links = [link.get_attribute("href") for link in links if isinstance(link.get_attribute("href"),str)]
                    urls = [link for link in links if len(find.findall(link)) > 0]
                    if len(urls) >num_links:
                        ingredients_dictionary[i] = urls[:num_links]
                    elif len(urls) == 0:
                        ingredients_dictionary[i] = None
                    else:
                        ingredients_dictionary[i] = urls
                else:
                    ingredients_dictionary[i] = [currentURL]
                

            except NoSuchElementException:
                print("could not find the search bar!")
                print(i)
                return ingredients_dictionary
    # close window
    finally:
        browser.close()
    # list with all the links
    return ingredients_dictionary


These links are collected in a dictionary with the ingredient as its respective value.

Then we are ready to get the nutritional value!

Continuing with our oats example, these are a few of the tables from which we got our nutrition data

![title](nutrition_values_oats.png)

In [ ]:
# code for scraper
def nutrition_value(dictionary,set_of_links = set()):
    """Takes in a dictionary with ingredients as keys
    look through the websites and scrape the nutritional value"""
    error_items =[]
    df_d = dict()
    browser = webdriver.Chrome()
    try:
        for k,v in zip(dictionary.keys(),dictionary.values()):
            if v is None:
                df_d[k] = {}
            else:    
                for l in v:
                    try:
                        if l is None:
                            print(f"No Website for: {k}")
                        elif l in set_of_links:
                            print(f"Duplicates for {k}")
                        else:
                            browser.get(l)
                            time.sleep(5)
                            # name of ingredient
                            name = browser.find_elements_by_tag_name('h1')[0]
                            name = name.text

                            #setting up nutritional values
                            nut = dict()
                            c = "tbody"
                            tables = browser.find_elements_by_tag_name(c)
                            #### For essentials [4]
                            ser_cal = tables[4].text.split('\n')
                            # Serving Size
                            nut[ser_cal[1][:12]] = ser_cal[1][13:]
                            #Calories
                            nut[ser_cal[3][:8]] = ser_cal[3][9:]

                            #### For all others [7-13]
                            n_v = re.compile('\s*(.*)\s([0-9]+\.[0-9]+\s\w+)')
                            for i in range(7,14):
                                nutrient_value = [n_v.findall(t) for t in tables[i].text.split('\n') if len(t) >0]
                                for t in nutrient_value:
                                    if len(t)>0:
                                        nut[t[0][0]] = t[0][1]

                            df_d[name] = nut
                            set_of_links.add(l)
                        
                    except IndexError as e:
                        error_items.append(k)
                        print(f"ingredient:{k}, error: {e}, link: {l}")
                    except:
                        error_items.append(k)
                        print(f"ingredient:{k}, error: IDK, link: {l}")
            
    finally:
        browser.close()
    # list with all the links
    df = pd.DataFrame.from_dict(df_d,'index')
    return df, error_items, set_of_links

The scraper would print out any errors it ran into and the link that caused it. Each link was inspected to make sure we weren't losing any important information. All printed out links were to other parts of the website that were gathered depending on the words we were searching. 

In order to not abuse the websites information we have also added a argument that skips the search if it has already been pulled.

Ones we collected the data we analysed it for errors. There were two columns that had only one value out of all the ingredients - the number "18" and "adjusted Protein". For this reason we dropped those columns. 

The rest of the data was cleaned by making all values floats, converting values to grams (g) and making serving sizes be 1g for all ingredients.

Vitamin A and D were a special case because they were measured in International Units (IU) so we converted them to grams

Lastly we engineered a columns for all minerals and vitamins for future investigation of trends in nutrition - Whether you can get all the nutrition you need from certain foods or recipes.

Code:

In [ ]:
# floats for calories
df = ds
df["Calories"] = df['Calories'].astype('float')/100
df = df.drop(columns = ["18","Adjusted Protein"]) # contains singleton

for j,c in enumerate(list(df.columns)):
    if c ==  "Calories":     
        pass
    else:    
        n = re.compile(r"(^\d*\.?\d*)\s(\w+)")     # float values
        
        ### changing Na for 0's, to go back change 0 to np.nan
        num = np.array([float(n.findall(i)[0][0]) if isinstance(i,str) else 0 for i in df[c].values])
        mes = np.array([n.findall(i)[0][1] if isinstance(i,str) else 'g' for i in df[c].values])

        # messurements
        mask_mg = (mes == 'mg')/10
        mask_mcg = (mes == "mcg")/10000
        mask_g = (mes == "g")*.01
        mask = mask_mg + mask_mcg +mask_g
        mask += (mask==0)*-1

        if sum(mask < 0) >1 :
            df[c] = num/100
        else:
            df[c] = num*mask

df["Vitamin A"] *= 0.6/1000000
df["Vitamin D"] *= 0.025/1000000

vitamins=['Choline','Niacin','Pantothenic acid','Riboflavin','Thiamin',
          'Vitamin A','Vitamin B12','Vitamin B6','Vitamin C','Vitamin D','Vitamin E','Vitamin K']
minerals = ['Calcium, Ca','Copper, Cu','Iron, Fe',    'Magnesium, Mg',
            'Manganese, Mn','Phosphorus, P','Potassium, K','Selenium, Se','Sodium, Na','Zinc, Zn']
# append new features
df['Vitamins'] = df[vitamins].sum(axis=1)
df['Minerals'] = df[minerals].sum(axis=1)
df.head()

Now that both data sets are clean and contain featured columns we merged them to form a giant sparse dataframe of the form *(see appendix)

One of our biggest challenges was to sort through the different quantities and converting them to grams. We have come up with a temporal solution why converts abstract measurements into the equivalent in grams for the most used ingredient. 

Example:

In [ ]:
conv = {c:[] for c in set(u.values())}
# g/ units
conv['gram'] = 1
conv['liter'] = 1000
conv["gallon"] = 0.00026417205
conv['lbs'] = 453.592
conv['milliliter'] = 1
conv['oz']= 28.34
conv['kilogram'] = 1000
conv['tablespoon'] = 14.3

We are continually searching for a solution to this problem. Nevertheless, We proceed with the analysis so that when an improvement to this method is found we can get better results. 

Code Quality and Robustness:

Data Visualization and Analysis:

# Apendix

### Code for hefty DataFrame

In [9]:
# gets mask for ingredients (CHECK MASK BEING USED)
col = list(neal_df.columns) 
mask_class = np.array([i for i in col if i[:6] == "class_"])

In [10]:

hefty_columns = [nut_element+"_"+category[6:] for category in mask_class for nut_element in df.columns]
hefty_columns += ["Total "+ nut_element for nut_element in df.columns]

In [ ]:
hefty_df = pd.DataFrame(columns=hefty_columns)
num_recipies = len(list(neal_df.index))
for num, recipe in enumerate(neal_df.index):
    r = neal_df.iloc[recipe]
    r_contents = r[mask_class].values != None
    ingredients = r[mask_class][r_contents].values
    hefty_df.loc[len(hefty_df)] = 0
    if num% 200 == 0:
        print(f"{num/num_recipies}%")
    for ing in ingredients:
        quantity = st_to_fl(r['quant_' + ing])
        unit_factor = conv[r['unit_' + ing]]
        conversion = quantity*unit_factor  
#         print(conversion)
        most_similar_ing = process.extractOne(ing,df.index)[0]
#         print(ing)
        col = list(hefty_df.columns) 
        hefty_mask = np.array([i for i in col if i[-(len(ing)+1):] == "_"+ing])
        for m,i,c in zip(hefty_mask, df.loc[most_similar_ing].values,df.columns):
            value = i*conversion
            hefty_df.loc[num][m] = value
            hefty_df.loc[num]["Total " + c] += value


### Code for making similar units the same by human input (for first time seen only)

In [ ]:
# changing units by hand
# u = dict()
# for i,rec in enumerate(neal_df[mask].values):
#     for j,mes in enumerate(rec):
#         if mes in u:
#             pass
#         else:
#             print(neal_df[mask].columns[j])
#             print(mes)
#             u[mes] = input()
#             print()